<a href="https://colab.research.google.com/github/bubu1435/bubu/blob/main/Smart_Sorting_Project_for_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# smart_sorting_app.py
# This script runs a Flask web server to host the Smart Sorting application
# within Google Colab, making it accessible via a public URL.

from flask import Flask, render_template_string, request, jsonify
import random
import os
import requests # For calling the Gemini API

# You might need to install flask and pyngrok if you haven't already
# !pip install Flask pyngrok

# Import ngrok for tunneling
from pyngrok import ngrok

# --- HTML Content for the Web Application ---
# The entire HTML structure, including CSS and JavaScript, is embedded here.
# This makes the Colab notebook self-contained.
HTML_CONTENT = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Smart Sorting: Rotten Fruits & Vegetables</title>
    <!-- Tailwind CSS CDN -->
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        /* Custom styles for Inter font and general body styling */
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f0f4f8; /* Light gray background */
            color: #334155; /* Darker text color */
        }
        .section-title {
            @apply text-2xl md:text-3xl font-bold text-indigo-700 mb-4 pb-2 border-b-2 border-indigo-300;
        }
        .subsection-title {
            @apply text-xl md:text-2xl font-semibold text-gray-800 mb-3;
        }
        .card {
            @apply bg-white rounded-lg shadow-md p-6 mb-6;
        }
        .button-primary {
            @apply bg-indigo-600 hover:bg-indigo-700 text-white font-bold py-2 px-4 rounded-full focus:outline-none focus:shadow-outline transition duration-300 ease-in-out;
        }
        .button-secondary {
            @apply bg-green-600 hover:bg-green-700 text-white font-bold py-2 px-4 rounded-full focus:outline-none focus:shadow-outline transition duration-300 ease-in-out;
        }
        .loading-spinner {
            border: 4px solid rgba(0, 0, 0, 0.1);
            border-left-color: #4f46e5;
            border-radius: 50%;
            width: 24px;
            height: 24px;
            animation: spin 1s linear infinite;
            display: inline-block;
            vertical-align: middle;
            margin-left: 8px;
        }
        @keyframes spin {
            to { transform: rotate(360deg); }
        }
    </style>
</head>
<body class="min-h-screen flex flex-col items-center py-8 px-4">

    <!-- Header Section -->
    <header class="w-full max-w-4xl text-center mb-12">
        <h1 class="text-4xl md:text-5xl font-extrabold text-indigo-800 mb-4">
            Smart Sorting: Transfer Learning For Identifying Rotten Fruits And Vegetables
        </h1>
        <p class="text-lg md:text-xl text-gray-600">
            An innovative project using AI for efficient waste reduction.
        </p>
    </header>

    <!-- Main Content Area -->
    <main class="w-full max-w-4xl">

        <!-- Project Overview Section -->
        <section id="overview" class="card">
            <h2 class="section-title">Project Overview</h2>
            <div class="space-y-4">
                <p>This project aims to develop a smart sorting system using transfer learning to accurately identify rotten fruits and vegetables. By leveraging pre-trained models, we can achieve high accuracy with less training data, contributing to food waste reduction.</p>
                <h3 class="subsection-title">Architecture</h3>
                <p>Details about the model architecture (e.g., CNN, specific pre-trained model like ResNet, VGG) will go here.</p>
                <h3 class="subsection-title">Prerequisites</h3>
                <ul class="list-disc list-inside ml-4 space-y-1">
                    <li>Python 3.x</li>
                    <li>TensorFlow/Keras</li>
                    <li>Numpy, Pandas, Matplotlib, Seaborn</li>
                    <li>Specific hardware requirements (e.g., GPU for faster training)</li>
                </ul>
                <h3 class="subsection-title">Prior Knowledge</h3>
                <ul class="list-disc list-inside ml-4 space-y-1">
                    <li>Basic understanding of Machine Learning and Deep Learning concepts.</li>
                    <li>Familiarity with Python programming.</li>
                    <li>Knowledge of image processing fundamentals.</li>
                </ul>
                <h3 class="subsection-title">Project Objectives</h3>
                <ul class="list-disc list-inside ml-4 space-y-1">
                    <li>Develop a robust model for classifying rotten and fresh produce.</li>
                    <li>Implement a user-friendly web application for real-time inference.</li>
                    <li>Achieve high accuracy and efficiency in sorting.</li>
                </ul>
                <h3 class="subsection-title">Project Flow</h3>
                <p>A high-level overview of the project's stages, from data collection to deployment.</p>
                <h3 class="subsection-title">Project Structure</h3>
                <p>Explanation of the directory structure (e.g., `data/`, `models/`, `app/`, `notebooks/`).</p>
            </div>
        </section>

        <!-- Data Collection And Preparation Section -->
        <section id="data-collection" class="card">
            <h2 class="section-title">Data Collection And Preparation</h2>
            <div class="space-y-4">
                <p>Machine Learning model performance heavily depends on data. This section outlines the process of gathering, cleaning, and preparing the dataset for training.</p>
                <h3 class="subsection-title">Collect The Dataset</h3>
                <p>Information on how the dataset was collected (e.g., public datasets, custom collection, web scraping) and its characteristics (e.g., size, types of fruits/vegetables).</p>
                <h3 class="subsection-title">Data Visualization</h3>
                <p>Details on exploratory data analysis (EDA) techniques used to understand the dataset, including sample images, class distribution, etc.</p>
                <h3 class="subsection-title">Data Augmentation</h3>
                <p>Explanation of augmentation techniques applied to increase dataset size and improve model generalization (e.g., rotation, flipping, zooming).</p>
            </div>
        </section>

        <!-- Split Data And Model Building Section -->
        <section id="model-building" class="card">
            <h2 class="section-title">Split Data And Model Building</h2>
            <div class="space-y-4">
                <p>This phase involves splitting the prepared data into training, validation, and test sets, followed by the actual construction and training of the machine learning model.</p>
                <h3 class="subsection-title">Model Building</h3>
                <p>Detailed description of the transfer learning approach, chosen pre-trained model, custom layers added, and training parameters (e.g., epochs, batch size, optimizer, loss function).</p>
            </div>
        </section>

        <!-- Testing Model & Data Prediction Section -->
        <section id="testing-prediction" class="card">
            <h2 class="section-title">Testing Model & Data Prediction</h2>
            <div class="space-y-4">
                <p>After training, the model's performance is evaluated using unseen data, and its ability to make accurate predictions is assessed.</p>
                <h3 class="subsection-title">Saving The Model</h3>
                <p>Instructions on how the trained model is saved for future use and deployment (e.g., HDF5, SavedModel format).</p>
            </div>
        </section>

        <!-- Application Building Section - Interactive Part -->
        <section id="application-building" class="card">
            <h2 class="section-title">Application Building - Try It Out!</h2>
            <div class="space-y-4">
                <p>Upload an image of a fruit or vegetable below to see a simulated prediction and get AI-powered tips!</p>

                <!-- Image Upload and Preview -->
                <div class="mb-4">
                    <label for="imageUpload" class="block text-gray-700 text-sm font-bold mb-2">Upload Image:</label>
                    <input type="file" id="imageUpload" accept="image/*" class="shadow appearance-none border rounded w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:shadow-outline" onchange="previewImage(event)">
                </div>
                <div class="mb-6 flex justify-center items-center h-48 bg-gray-100 rounded-md overflow-hidden border border-gray-300">
                    <img id="imagePreview" src="https://placehold.co/200x200/E0E7FF/4338CA?text=Image+Preview" alt="Image Preview" class="max-w-full max-h-full object-contain">
                </div>

                <!-- Prediction Button -->
                <div class="flex justify-center space-x-4 mb-6">
                    <button id="predictButton" class="button-primary">
                        Predict
                    </button>
                </div>

                <!-- Prediction Result -->
                <div id="predictionResult" class="mt-4 text-center text-lg font-semibold"></div>
                <div id="loadingSpinner" class="hidden text-center mt-2">
                    <div class="loading-spinner"></div>
                    <span class="ml-2 text-gray-600">Thinking...</span>
                </div>

                <!-- Gemini API Powered Tips Section -->
                <div id="geminiTipsSection" class="hidden mt-6 p-4 bg-blue-50 rounded-lg border border-blue-200">
                    <h3 class="subsection-title text-blue-800">AI-Powered Tips ✨</h3>
                    <div class="flex flex-wrap justify-center gap-4 mb-4">
                        <button id="getStorageTipsButton" class="button-secondary hidden">
                            ✨ Get Storage Tips ✨
                        </button>
                        <button id="getRecipeTipsButton" class="button-secondary hidden">
                            ✨ Get Recipe Ideas ✨
                        </button>
                        <button id="getNutritionalInfoButton" class="button-secondary hidden">
                            ✨ Get Nutritional Info ✨
                        </button>
                        <button id="getCompostingTipsButton" class="button-secondary hidden">
                            ✨ Get Composting Tips ✨
                        </button>
                        <button id="getDisposalTipsButton" class="button-secondary hidden">
                            ✨ Get Disposal Guidance ✨
                        </button>
                        <button id="getDiyIdeasButton" class="button-secondary hidden">
                            ✨ Get DIY Ideas ✨
                        </button>
                    </div>
                    <div id="tipsContent" class="text-gray-700 leading-relaxed"></div>
                </div>

            </div>
        </section>

    </main>

    <!-- Footer Section -->
    <footer class="w-full max-w-4xl text-center mt-12 py-6 border-t-2 border-gray-200 text-gray-500 text-sm">
        <p>&copy; 2025 Smart Sorting Project. All rights reserved.</p>
    </footer>

    <script>
        // Global variable to store the last predicted produce type and status
        let lastPrediction = { status: '', type: '' };

        // Function to preview the uploaded image
        function previewImage(event) {
            const reader = new FileReader();
            reader.onload = function() {
                const output = document.getElementById('imagePreview');
                output.src = reader.result;
                // Reset prediction and tips when a new image is uploaded
                document.getElementById('predictionResult').innerText = '';
                document.getElementById('geminiTipsSection').classList.add('hidden');
                document.getElementById('getStorageTipsButton').classList.add('hidden');
                document.getElementById('getRecipeTipsButton').classList.add('hidden');
                document.getElementById('getNutritionalInfoButton').classList.add('hidden');
                document.getElementById('getCompostingTipsButton').classList.add('hidden');
                document.getElementById('getDisposalTipsButton').classList.add('hidden');
                document.getElementById('getDiyIdeasButton').classList.add('hidden');
                document.getElementById('tipsContent').innerText = '';
            };
            reader.readAsDataURL(event.target.files[0]);
        }

        // Simulate a prediction (replace with actual ML model inference later)
        // For demonstration, it will randomly pick 'rotten apple' or 'fresh banana'
        function simulatePrediction() {
            const produceTypes = ['apple', 'banana', 'orange', 'grape', 'tomato', 'potato', 'carrot', 'broccoli'];
            const randomProduce = produceTypes[Math.floor(Math.random() * produceTypes.length)];
            const isRotten = Math.random() < 0.5; // 50% chance of being rotten
            return {
                status: isRotten ? 'rotten' : 'fresh',
                type: randomProduce
            };
        }

        // Main prediction logic
        document.getElementById('predictButton').addEventListener('click', () => {
            const fileInput = document.getElementById('imageUpload');
            if (!fileInput.files || fileInput.files.length === 0) {
                document.getElementById('predictionResult').innerText = 'Please upload an image first!';
                return;
            }

            document.getElementById('loadingSpinner').classList.remove('hidden');
            document.getElementById('predictionResult').innerText = '';
            document.getElementById('geminiTipsSection').classList.add('hidden');
            document.getElementById('getStorageTipsButton').classList.add('hidden');
            document.getElementById('getRecipeTipsButton').classList.add('hidden');
            document.getElementById('getNutritionalInfoButton').classList.add('hidden');
            document.getElementById('getCompostingTipsButton').classList.add('hidden');
            document.getElementById('getDisposalTipsButton').classList.add('hidden');
            document.getElementById('getDiyIdeasButton').classList.add('hidden');
            document.getElementById('tipsContent').innerText = '';

            // Simulate API call delay
            setTimeout(() => {
                const prediction = simulatePrediction();
                lastPrediction = prediction; // Store the prediction globally
                document.getElementById('loadingSpinner').classList.add('hidden');
                document.getElementById('predictionResult').innerText = `Prediction: This looks like a ${prediction.status} ${prediction.type}!`;

                document.getElementById('geminiTipsSection').classList.remove('hidden');

                // Show/hide buttons based on prediction status
                if (prediction.status === 'rotten') {
                    document.getElementById('getCompostingTipsButton').classList.remove('hidden');
                    document.getElementById('getDisposalTipsButton').classList.remove('hidden');
                    document.getElementById('getDiyIdeasButton').classList.remove('hidden'); // Show DIY button
                    document.getElementById('getStorageTipsButton').classList.add('hidden');
                    document.getElementById('getRecipeTipsButton').classList.add('hidden');
                    document.getElementById('getNutritionalInfoButton').classList.add('hidden');
                } else { // fresh
                    document.getElementById('getStorageTipsButton').classList.remove('hidden');
                    document.getElementById('getRecipeTipsButton').classList.remove('hidden');
                    document.getElementById('getNutritionalInfoButton').classList.remove('hidden'); // Show Nutritional Info button
                    document.getElementById('getCompostingTipsButton').classList.add('hidden');
                    document.getElementById('getDisposalTipsButton').classList.add('hidden');
                    document.getElementById('getDiyIdeasButton').classList.add('hidden');
                }
            }, 1500); // Simulate network delay
        });

        // Event listeners for the new tip buttons
        document.getElementById('getStorageTipsButton').addEventListener('click', () => {
            getGeminiTips(lastPrediction.type, 'storage');
        });

        document.getElementById('getRecipeTipsButton').addEventListener('click', () => {
            getGeminiTips(lastPrediction.type, 'recipe');
        });

        document.getElementById('getNutritionalInfoButton').addEventListener('click', () => {
            getGeminiTips(lastPrediction.type, 'nutritional');
        });

        document.getElementById('getCompostingTipsButton').addEventListener('click', () => {
            getGeminiTips(lastPrediction.type, 'composting');
        });

        document.getElementById('getDisposalTipsButton').addEventListener('click', () => {
            getGeminiTips(lastPrediction.type, 'disposal');
        });

        document.getElementById('getDiyIdeasButton').addEventListener('click', () => {
            getGeminiTips(lastPrediction.type, 'diy');
        });


        // Function to call Gemini API for tips
        async function getGeminiTips(produceType, tipType) {
            document.getElementById('tipsContent').innerText = 'Generating tips...';
            document.getElementById('loadingSpinner').classList.remove('hidden'); // Show spinner for Gemini call

            let prompt = '';
            if (tipType === 'composting') {
                prompt = `Provide concise and practical composting tips for a rotten ${produceType}. Focus on what to do and what to avoid.`;
            } else if (tipType === 'storage') {
                prompt = `Provide concise and practical storage tips for a fresh ${produceType} to extend its shelf life.`;
            } else if (tipType === 'recipe') {
                prompt = `Suggest 3-5 creative and easy recipe ideas using fresh ${produceType}. Provide brief descriptions for each.`;
            } else if (tipType === 'disposal') {
                prompt = `Provide concise and practical disposal guidance for a rotten ${produceType}. Consider options beyond composting if applicable.`;
            } else if (tipType === 'nutritional') {
                prompt = `Provide key nutritional information for a fresh ${produceType}, including calories, main vitamins, and minerals.`;
            } else if (tipType === 'diy') {
                prompt = `Suggest 3-5 creative DIY or craft ideas using a rotten ${produceType}. Focus on safe and practical uses.`;
            }

            let chatHistory = [];
            chatHistory.push({ role: "user", parts: [{ text: prompt }] });

            const payload = { contents: chatHistory };
            // In Colab, we'll proxy this request through Flask to handle the API key securely.
            // The frontend will call a Flask endpoint, which then calls the Gemini API.
            const apiUrl = '/gemini-proxy';

            try {
                const response = await fetch(apiUrl, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ prompt: prompt }) // Send the prompt to the Flask backend
                });
                const result = await response.json();

                if (result.text) { // Flask backend sends 'text'
                    document.getElementById('tipsContent').innerHTML = result.text.replace(/\n/g, '<br>');
                } else {
                    document.getElementById('tipsContent').innerText = 'Could not retrieve tips. Please try again.';
                    console.error('Gemini API response structure unexpected:', result);
                }
            } catch (error) {
                document.getElementById('tipsContent').innerText = 'Error fetching tips. Please check your network connection or the Colab server.';
                console.error('Error calling Gemini API proxy:', error);
            } finally {
                document.getElementById('loadingSpinner').classList.add('hidden'); // Hide spinner
            }
        }
    </script>
</body>
</html>
"""

app = Flask(__name__)

# --- Flask Routes ---

@app.route('/')
def index():
    """Serves the main HTML content of the application."""
    return render_template_string(HTML_CONTENT)

@app.route('/gemini-proxy', methods=['POST'])
def gemini_proxy():
    """
    Proxies requests to the Gemini API.
    This is necessary in Colab to keep the API key secure on the backend.
    """
    data = request.get_json()
    prompt = data.get('prompt')

    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400

    # Retrieve API key from environment variables for security
    # IMPORTANT: You MUST set this environment variable in your Colab notebook
    # before running the Flask app.
    # Example: os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY_HERE"
    gemini_api_key = os.environ.get("GEMINI_API_KEY")

    if not gemini_api_key:
        return jsonify({'error': 'Gemini API key not configured on the server.'}), 500

    gemini_payload = {
        "contents": [{"role": "user", "parts": [{"text": prompt}]}]
    }
    gemini_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={gemini_api_key}"

    try:
        gemini_response = requests.post(
            gemini_url,
            headers={'Content-Type': 'application/json'},
            json=gemini_payload
        )
        gemini_response.raise_for_status() # Raise an exception for HTTP errors
        gemini_result = gemini_response.json()

        if gemini_result.get('candidates') and gemini_result['candidates'][0].get('content') and \
           gemini_result['candidates'][0]['content'].get('parts') and \
           gemini_result['candidates'][0]['content']['parts'][0].get('text'):
            return jsonify({'text': gemini_result['candidates'][0]['content']['parts'][0]['text']})
        else:
            return jsonify({'error': 'Unexpected Gemini API response structure.'}), 500

    except requests.exceptions.RequestException as e:
        return jsonify({'error': f'Error calling Gemini API: {e}'}), 500
    except Exception as e:
        return jsonify({'error': f'An unexpected error occurred: {e}'}), 500

# --- Running the Flask app with ngrok ---
def run_flask_app():
    """Starts the Flask app and exposes it via ngrok."""
    # Set your ngrok authtoken here or via environment variable
    # ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN") # Uncomment and set if you have one

    # Open a HTTP tunnel on port 5000
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel available at: {public_url}")
    print(" * Open this URL in your browser to access the app.")

    # Run Flask app
    app.run(port=5000, debug=False, use_reloader=False) # use_reloader=False is important for Colab

if __name__ == '__main__':
    # This block will be executed when the script is run directly.
    # In a Colab notebook, you would typically call run_flask_app()
    # after setting up your API keys.
    print("To run this in Google Colab, execute the following steps in a cell:")
    print("1. Install necessary libraries:")
    print("   !pip install Flask pyngrok requests")
    print("2. Set your Gemini API Key as an environment variable (replace YOUR_API_KEY):")
    print("   import os")
    print("   os.environ['GEMINI_API_KEY'] = 'YOUR_GEMINI_API_KEY'")
    print("   # If you have an ngrok authtoken (optional, but good for stable URLs):")
    print("   # !ngrok authtoken YOUR_NGROK_AUTHTOKEN")
    print("3. Copy and paste the entire Python code block above into a new cell.")
    print("4. Call the function to run the app:")
    print("   run_flask_app()")
    print("\nOnce running, look for the 'ngrok tunnel available at:' URL and open it.")

In [2]:
!pip install Flask pyngrok requests